### Travelling Salesman Problem (TSP): Exercise 1 - Understanding The Code - Visiting UK Airports

#### Useful Links and Documentation
- [`Path.MOVETO`](https://matplotlib.org/stable/tutorials/advanced/path_tutorial.html)
- [GeoPy](https://geopy.readthedocs.io/en/stable/)
- ['Hill Climbing'](https://www.geeksforgeeks.org/introduction-hill-climbing-artificial-intelligence/)

#### Introduction

The travelling salesman problem is a classic geographic problem that may be framed as "Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city and returns to the origin city?".

#### Background

This problem is a class of problems known as *'np-hard'*, that is the problem size grows non-linearly with the number of points to visit, and soon becomes too computationally expensive to compute **all** possibilities. There is also no algorithm known that efficiently gives a certain optimum solution. The table below illustrates the speed with which the number of possible routes increases as the number of points to visit grows.

| Points | No. Possible Routes        |
|--------|----------------------------|
| 5      | 120                        |
| 10     | 3,628,800                  |
| 20     | 2,432,902,008,176,640,000  |

Np-hard problems are ‘solved’ by using a *‘heuristic algorithm’*. Heuristic algorithms follow a method designed to efficiently give a sufficiently good solution at the expense of not guaranteeing that an optimal solution is found.

In the case of the travelling salesman problem there are many heuristics described. It is a favourite problem of algorithm writers!

In the code below we will use a ‘hill-climbing’ method based on reversing portions of the route (or a ‘pairwise exchange’ approach). Hill-climbing methods perform some search that then leads to them picking the best improvement in that step of the search. The search method is then repeated until no further improvement is found. Hill-climbing methods may get trapped in local optima in complicated problems – the solution found depends on the starting point. To reduce the impact of this the algorithm is repeated multiple times with different start points.

#### Steps to be Performed

Our algorithm will follow these steps:

**Step 1.** Pick a route at random (randomly order points to visit).

**Step 2.** Examine all possible pairwise exchanges in the route in (e.g. if we have a route A-B-<span style="color:blue">**C**</span>-D-E-<span style="color:blue">**F**</span>-G-H, and pairwise exchange **C** and **F** we have a new route A-B-<span style="color:red">**F**</span>-E-D-<span style="color:red">**C**</span>-G-H). Choose the pairwise exchange that gives the best reduction in route distance).

**Step 3.** Repeat **Step 2.** until no further improvement is observed.

**Step 4.** Repeat from **Step 1** for a determined number of repeats (or repeat until a maximum algorithm use time is reached).

In order to run an algorithm like this we need to know the travel times or distances between points. In the example below we calculate a straight line (or ‘[Euclidean](https://en.wikipedia.org/wiki/Euclidean_distance)’) distance based on x and y locations. x/y locations are readily available for addresses (such as Eastings and Northings in the UK, or Lat/ Longs). Straight line distances or travel times, while only approximate may be good enough to get reasonable solutions. More precise solutions could use travel times or distances determined from Geographical Information Systems. You can watch a video using one such application, Routino on [Windows Subsystem for Linux](https://en.wikipedia.org/wiki/Windows_Subsystem_for_Linux) (WSL) [here](https://www.youtube.com/watch?v=y-RgaPDnjvI).

In [ ]:
# Required Library Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
import math
import geopy.distance
import pandas as pd

In [ ]:
def calculate_total_route_distance(route, distances_lookup):
    """Helper function to calculate route distances. Accesses 'distances_lookup' 
    dictionary which gives distance between any two points. Total distance 
    includes return to starting point at end of route
    
    Params
    ------
    route : list
        List of points to visit (stored as index numbers)
    
    distances_lookup : dict
        Dictionary of distances for X, Y point pairs
    
    Output
    ------
    total : float
        Total distance for route
    
    """

    total = 0
    for i in range(len(route) - 1):
        total += distances_lookup[route[i], route[i + 1]]
    # return to starting point below.
    total += distances_lookup[route[-1], route[0]]
    return total

def reverse_section(orig_list, point_a, point_b):
    """Reverses section of route between points a and b (inclusive)
    
    Params
    ------
    orig_list : list
        Routse list
        
    point_a : int
        Index number of Point A
    
    point_b : int
        Index number of Point B    
    
    Output
    ------
    
    new_list : list
        Updated list
    """

    low_switch_point = min(point_a, point_b)
    high_switch_point = max(point_a, point_b)
    high_switch_point += 1  # Include high switch point in reversed section due to way slicing works
    section_1 = orig_list[0:low_switch_point]
    section_2 = list(reversed(orig_list[low_switch_point:high_switch_point]))
    section_3 = orig_list[high_switch_point:]
    new_list = section_1 + section_2 + section_3
    return (new_list)

def find_shortest_route(runs, points_to_visit, distances_lookup):
    """Main algorithm code to find the shortest distances
    
    
    Params
    ------
    runs : int
        Number of attempts to find the shortest distance
    
    points_to_visit : list
        List of points to visit (stored as index numbers)
        
    distances_lookup : dict
        Dictionary of distances for X, Y point pairs.
               
    Outputs
    -------
    final_best_distance_so_far : float
        Shortest distance
    
    final_best_route_so_far : list
        Best/ shortest distances achieved
    
    progress_in_best_distance : list
        List of all final_best_distance_so_far
    
    """
    
    final_best_route_so_far = []
    # le99 is a way of representing a large number so even first result/ lookup...
    # will be less than it... c.f. infinity
    final_best_distance_so_far = 1e99
    progress_in_best_distance = []

    
    for run in range(runs):
        exchange_point_1 = 0  
        exchange_point_2 = 0  
        improvement_found = True
        best_route_so_far = points_to_visit.copy()  
        np.random.shuffle(best_route_so_far)
        best_distance_so_far = calculate_total_route_distance(
                best_route_so_far, distances_lookup)

        while improvement_found:  # continue until no further improvement
            improvement_found = False

            # Loop through all pairwise route section reversals
            for i in range(0, len(best_route_so_far)-1):
                for j in range(i, len(best_route_so_far)):
                    test_route = best_route_so_far.copy()
                    test_route = reverse_section(test_route, i, j)
                    test_route_distance = (calculate_total_route_distance
                                (test_route, distances_lookup))
                    if test_route_distance < best_distance_so_far:
                        exchange_point_1 = i
                        exchange_point_2 = j
                        improvement_found = True
                        best_distance_so_far = test_route_distance

            if improvement_found:
                best_route_so_far = reverse_section(
                        best_route_so_far, exchange_point_1, exchange_point_2)
   
        if best_distance_so_far < final_best_distance_so_far:
            final_best_distance_so_far = best_distance_so_far
            final_best_route_so_far = best_route_so_far.copy()

        progress_in_best_distance.append(final_best_distance_so_far)

    return (final_best_distance_so_far, final_best_route_so_far, 
            progress_in_best_distance)

def plot_improvement(progress_in_best_distance):
    """Helper function to plot improvement over runs i.e., y axis is the fitness, x is the run number
    
    Param
    -----
    progress_in_best_distance : list
        List of results achieved during each run
    """
    
    plt.plot(progress_in_best_distance)
    plt.xlabel('Run')
    plt.ylabel('Best distance')
    plt.show()
    
def create_dictionary_of_distance_airport(df):
    """Helper function to create a dictionary, `distances_lookup` between all 
    combinations of `points_to_visit`. Used GeoPy library to assist.
    
    Params
    ------
    df : pd.DataFrame
        Details of airport locations
           
    Output
    ------
    distances_lookup : dict
        Dictionary of distances for X, Y point pairs.
        
    airport_lookup : dict
        Dictionary of index to 3 char IATA airport code
    """
    
    distances_lookup = {}
    airport_lookup = {}
    number_of_points_to_visit = df.shape[0]

    for i in range(number_of_points_to_visit):
        for j in range(i, number_of_points_to_visit):
            coords_1 = (df.Lat.iloc[i], df.Long.iloc[i])
            coords_2 = (df.Lat.iloc[j], df.Long.iloc[j])
            distance = geopy.distance.geodesic(coords_1, coords_2).miles
            distances_lookup[i, j] = distance
            distances_lookup[j, i] = distance
        airport_lookup[i] = df.Code.iloc[i]

    return distances_lookup, airport_lookup

def print_results_airport(final_best_route_so_far, xCo, yCo, points_to_visit, 
                   final_best_distance_so_far, progress_in_best_distance, airport_lookup):
    """Prints results to screeen
    
    Params
    ------
    final_best_route_so_far : list
        Best/ shortest distances achieved
    
    xCo : np.array
        Array for n ints that form the X-coords
    
    yCo : np.array
        Array for n ints that form the Y-coords
    
    points_to_visit : list
        List of points to visit   
                   
    final_best_distance_so_far : int
        Shortest distance : int
    
    progress_in_best_distance : list
        List of all final_best_distance_so_far
        
    airport_lookup : dict
        Dictionary of index to airport IATA codes
    """
    
    print('Best route found:')
    plot_route_airport(final_best_route_so_far, xCo, yCo, points_to_visit, airport_lookup)
    print('\nDistance = %.0f' % final_best_distance_so_far)
    print('\nImprovement in distance over run:')
    plot_improvement(progress_in_best_distance)
    
def plot_route_airport(route, xCo, yCo, label, airport_lookup):
    """Helper function to plot points and best route found between points
    
    Params
    ------
    route : List
        Route by index numbers
    
    xCo : np.array
        Array for n ints that form the X-coords
    
    yCo : np.array
        Array for n ints that form the Y-coords
        
    label :  list
        List of points to visit i.e., names
        
    airport_lookup : dict
        Dictionary of index to airport IATA codes
    """
    
    # Create figure
    fig = plt.figure(figsize=(12, 6))

   # Plot airport locations
    ax1 = fig.add_subplot(121)
    ax1.scatter(xCo, yCo)
    texts = []
    for i, txt in enumerate(label):
        txt = airport_lookup.get(txt)
        texts.append(ax1.text(xCo[i] + 0.1, yCo[i] + 0.1, txt))

    ax1.set_xlim(min(xCo) - 1, max(xCo) + 1)
    ax1.set_ylim(min(yCo) - 1, max(yCo) + 1)  
        
    # Plot best route found between points
    verts = [None] * int(len(route) + 1)
    codes = [None] * int(len(route) + 1)
    for i in range(0, len(route)):
        verts[i] = xCo[route[i]], yCo[route[i]]
        if i == 0:
            codes[i] = Path.MOVETO
        else:
            codes[i] = Path.LINETO
    verts[len(route)] = xCo[route[0]], yCo[route[0]]
    codes[len(route)] = Path.CLOSEPOLY

    path = Path(verts, codes)

    ax2 = fig.add_subplot(122)
    patch = patches.PathPatch(path, facecolor='none', lw=0)
    ax2.add_patch(patch)

    ax2.set_xlim(min(xCo) - 1, max(xCo) + 1)
    ax2.set_ylim(min(yCo) - 1, max(yCo) + 1)

    # give the points a label
    xs, ys = zip(*verts)
    ax2.plot(xs, ys, 'x--', lw=2, color='black', ms=10)

    texts = []
    for i, txt in enumerate(label):
        #txt = airport_lookup.get(txt)
        txt = airport_lookup.get(txt)
        texts.append(ax2.text(xCo[i] + 0.1, yCo[i] + 0.1, txt))

    # Display plot    
    plt.tight_layout(pad=4)
    plt.show()
    return

In [ ]:
def main_code_airport(number_of_runs=50):
    """Main program code called.
    
    Params
    ------
    number_of_runs : int (default = 50)
        Number of runs/ attempts"""
    #Assigning variable names using the imported dataframe ('airports')
    points_to_visit = [i for i in range(airports.shape[0])]
    yCo = airports.Lat.to_list()
    xCo = airports.Long.to_list()
    number_of_points_to_visit = airports.shape[0]

    # Creating distance dictionary and lookup~
    distances_lookup, airport_lookup = create_dictionary_of_distance_airport(airports)

    # Run route finding algorithm multiple times
    (final_best_distance_so_far, final_best_route_so_far, 
     progress_in_best_distance) = (find_shortest_route(
             number_of_runs, points_to_visit, distances_lookup))

    
    # Print results     
    print_results_airport(final_best_route_so_far, xCo, yCo, points_to_visit, 
                   final_best_distance_so_far, progress_in_best_distance, airport_lookup)


In [ ]:
airports = pd.read_csv('../data/airports.csv')

In [ ]:
# Need to add the Univserity as the starting address
new_row = ["University", 'UoE', 50.7224, -3.5166]
airports.loc[len(airports)] = new_row

In [ ]:
main_code_airport()